# Some simple forecasting methods

## Average method
The forecast of all future values are equal to the average of the historical data. If we let the historical data be denoted by $y_1, ...., y_T$, then we can write the forecast as 

$\hat{y}_{T+h|T}=\overline{y}=(y_1+...+y_T)/T$

## Naive method
For naive forecasts, we simply set all forecasts to be the value of the last observation. That is, 

$\hat{y}_{T+h|T}=y_T$

This method works remarkably well for many economic and finanical time series. Because a naive forecast is optimal when data follows a random walk. 

## Seasonal naive method
We set each forecast to be equal to the last observed value from the same season of the year. Formally, the forecast for time $T+h$ is written as 
$\hat{y}_{T+h|T}=y_{T+h-m(k+1)}$
where $m =$ the seasonal period, and $k$ is the integer part of $(h-1)/m$ the number of complete year in the forecast period prior to time $T+h$.

## Drift method
A variation on the naive method is to allow the forecasts to increase or decrease over time, where the amount of change over time (called the drift) is set to be the averge change seen in the historical data. 

$\hat{y}_{T+h|T} = y_T + h(\frac{y_T-y_1}{T-1})$

# Transformations and adjustments
